<a href="https://colab.research.google.com/github/chanlenium/Python/blob/main/CIS%20eFAS%20Industry%20DB%20Scraping/IT_EFAS_PPI_INDEX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##### 조회년월
queryYYYYMM = '202211'

##### 조회분기
queryYYYYQQ = '2022Q3'

In [ ]:
import requests
import pandas as pd
from datetime import date

key = 'MTSA07N58C5X8EB4LMJL'

def ecosApiCall(url_tail):
  url = f'https://ecos.bok.or.kr/api/StatisticSearch/{key}/json/kr/1/1000/{url_tail}'
  r = requests.get(url)
  jo = r.json()
  df = pd.DataFrame(jo['StatisticSearch']['row'])
  df = df[['TIME', 'DATA_VALUE']]
  df.rename(columns = {'TIME' : 'STD_YM'}, inplace = True)
  df['DATA_VALUE'] = pd.to_numeric(df['DATA_VALUE'])
  df['STD_YM'] = df['STD_YM'].str.replace('Q1$', '03', regex=True)
  df['STD_YM'] = df['STD_YM'].str.replace('Q2$', '06', regex=True)
  df['STD_YM'] = df['STD_YM'].str.replace('Q3$', '09', regex=True)
  df['STD_YM'] = df['STD_YM'].str.replace('Q4$', '12', regex=True)
  df = df.set_index('STD_YM')
  return df

# (1) 생산자물가지수(월별)
# 4.1.1.1. 생산자물가지수(기본분류) : [404Y014][M]   /   총지수 : [*AA][2015=100]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/404Y014/Q/2010Q1/2020Q4/*AA
monthlyPPI = ecosApiCall(f"404Y014/M/201001/{queryYYYYMM}/*AA")
monthlyPPI.insert(0, 'GBN', '1')
monthlyPPI.rename(columns = {'DATA_VALUE':'PPI'}, inplace = True)

# (2) 생산자물가지수(월별)
# 4.1.1.1. 생산자물가지수(기본분류) : [404Y014][Q]   /   총지수 : [*AA][2015=100]
# (Ex) https://ecos.bok.or.kr/api/StatisticSearch/MTSA07N58C5X8EB4LMJL/json/kr/1/100/404Y014/Q/2010Q1/2020Q4/*AA
quaterlyPPI = ecosApiCall(f"404Y014/Q/2010Q1/{queryYYYYQQ}/*AA")
quaterlyPPI.insert(0, 'GBN', '2')
quaterlyPPI.rename(columns = {'DATA_VALUE':'PPI'}, inplace = True)

PPI_INDEX = pd.concat([monthlyPPI, quaterlyPPI], ignore_index = False)
PPI_INDEX = PPI_INDEX.sort_index()

today = date.today()
PPI_INDEX.to_excel(f'{today}_PPI_INDEX.xlsx')
PPI_INDEX

,GBN,PPI
STD_YM,,
201001,1,96.71
201002,1,96.88
201003,2,96.97
201003,1,97.33
201004,1,98.04
...,...,...
202207,1,120.44
202208,1,119.98
202209,1,120.06
